## Count-Min Sketching

In [1]:
import pandas as pd
import numpy as np
import time
import random

## Data preparation

In [2]:
data_file = '../data/capture20110811.pcap.netflow_43_.labeled'
infected = '147.32.84.165'
n = 10
with open(data_file, "r") as ins:
    lines = ins.readlines()

## Perform sketching

In [3]:
# define various values for the number of hash functions and their range
ds = [50,200,25]
ws = [500,1000,700]
# run 10 iterations to average the run-time
iterations = 10
run_times = np.zeros([iterations,len(ds)])


for it in range(iterations):
    estimates = []
    
    for i in range(len(ds)):
        d = ds[i]
        w = ws[i]
        total = 0
        # dictionaries to facilitate IP to id translation and vice versa
        ip_dict = {}
        rev_ip_dict = {}

        # initialize count min sketch as an array of zeros
        cm = np.zeros((d,w))
        k = 0

        coefs = []
        consts = []

        # use the hash function from the slides
        def hash_func(coef,const,base,value):
            return (coef*value + const)%base

        # start time recording
        start = time.time()

        # create independent hash functions by using a different coefficient and bias term for each
        for j in range(d):
            temp = random.randint(1,d)
            while temp in coefs:
                temp = random.randint(1,d)
            coefs.append(temp)
            temp = random.randint(1, d)
            while temp in consts:
                temp = random.randint(1,d)
            consts.append(temp)


        for line in lines:
            parts = line.split()
            ip_port_src = parts[4].split(':')
            ip_src = ip_port_src[0]
            
            # if this is from our infected host
            if ip_src == infected:
                total += 1
                ip_port_dst = parts[6].split(':')
                dst_ip = ip_port_dst[0]
                
                # find the id of the IP
                if dst_ip not in ip_dict.keys():
                    ip_dict[dst_ip] = k
                    rev_ip_dict[k] = dst_ip
                    k += 1
                temp = ip_dict[dst_ip]
                # use the id to get a hash from each hash function and update the sketch
                for j in range(d):
                    col = hash_func(coefs[j], consts[j], w, temp)
                    cm[j, col] = cm[j, col] + 1

        # find the minimum value for each IP
        A = np.zeros((k - 1, 1), dtype=np.int32)
        for l in range(k - 1):
            minimum = total
            for j in range(d):
                temp = cm[j][hash_func(coefs[j], consts[j], w, l)]
                if temp < minimum:
                    A[l] = temp
                    minimum = temp

        # find the 10 most frequent IPs
        out = A.flatten()
        res = np.argsort(out)
        ips_estimated = []
        counts = []

        
        # stop time recording
        stop = time.time()
        run_times[it][i] = stop - start
        
        for j in res[-10:]:
            ips_estimated.append(rev_ip_dict[j])
            counts.append(round(out[j]/total,3))
        

        # store results for evaluation
        estimates.append({
            'ips': np.array(ips_estimated),
            'freqs': np.array(counts),
        })

## Figure out true top 10

In [4]:
# calculate the actual counts of each IP
ips = {}
infected_flow_count = 0
n = 10
with open("../data/capture20110811.pcap.netflow_43_.labeled", "r") as ins:
    for line in ins:
        parts = line.split()
        ip_port_src = parts[4].split(':')    
        ip_src = ip_port_src[0]
        if ip_src == infected:
            ip_port_dst = parts[6].split(':')
            ip_dst = ip_port_dst[0]
            if not ip_dst in ips:
                ips[ip_dst] = 0
            ips[ip_dst] += 1
            infected_flow_count += 1

ips_ip = np.array(list(ips.keys()))
ips_count = np.array(list(ips.values()))

# keep the top 10 IPs
ind = np.argsort(-ips_count)[:n]
true = {}
true['ips'] = ips_ip[ind]
true['freqs'] = ips_count[ind] / infected_flow_count

## Build estimate table

In [5]:
for i in range(n):
    line = [i+1, true['ips'][i], round(true['freqs'][i],3)]
    for j,k in enumerate(ws):
        line.append(estimates[j]['ips'][-i-1])
        line.append(round(estimates[j]['freqs'][-i-1],3))
    print("\t".join([str(x) for x in line]))

1	193.23.181.44	0.136	62.80.17.242	0.136	193.23.181.44	0.136	193.23.181.44	0.136
2	174.128.246.102	0.076	200.143.5.118	0.136	209.59.172.38	0.136	168.61.70.66	0.136
3	174.37.196.55	0.074	209.59.172.38	0.136	174.128.246.102	0.076	64.85.60.138	0.136
4	67.19.72.206	0.069	193.23.181.44	0.136	204.11.209.99	0.076	83.238.197.86	0.076
5	72.20.15.61	0.066	174.128.246.102	0.076	174.37.196.55	0.074	200.42.208.2	0.076
6	173.236.31.226	0.038	204.11.209.99	0.076	67.69.240.18	0.074	174.128.246.102	0.076
7	184.154.89.154	0.037	64.182.71.51	0.076	67.19.72.206	0.069	174.37.196.55	0.074
8	46.4.36.120	0.036	130.220.2.1	0.076	63.209.10.244	0.069	95.172.94.59	0.074
9	147.32.80.9	0.017	85.158.228.111	0.074	72.20.15.61	0.066	66.210.5.50	0.074
10	217.163.21.37	0.015	174.37.196.55	0.074	209.223.88.11	0.066	74.125.232.199	0.069


## Recall

In [6]:
'''
freq_distance measures the distance between the frequencies in the 
ground-truth list and the frequencies in an estimated list.

true, estimate: objects with two list .ips and .freqs
'''
def freq_distance(true, estimate):
    estimate_map = {}
    for i, ip in enumerate(estimate['ips']):
        estimate_map[ip] = estimate['freqs'][i]
    
    score = 0
    for i, ip in enumerate(true['ips']):
        if ip in estimate_map:
            score += abs(true['freqs'][i] - estimate_map[ip])
        else:
            score += true['freqs'][i]
    return score   

In [7]:
# create output for recall table
for j in range(len(estimates)):
    recall = float(len(np.intersect1d(true['ips'], estimates[j]['ips']))) / float(n)
    freq_score = freq_distance(true, estimates[j])
    run_time = round(np.mean(run_times,axis=0)[j],3)
    print("{}\t{}\t{}\t{}".format((ws[j],ds[j]), recall, round(freq_score,4),run_time))

(500, 50)	0.3	0.2791	7.416
(1000, 200)	0.5	0.1451	13.488
(700, 25)	0.3	0.2791	6.588
